### Engineering

In [1]:
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
api server, scheduler, control manager, ectd

In [ ]:
#include <iostream>

In [ ]:
using namespace std;

In [ ]:
class Book() {
    public:
        string title;
        string author;
}

In [ ]:
monitor
if a node failed, reassign tasks for surviving nodes
if a new node enters the cluster, adjust communication ring

In [ ]:
step 1: scale the loss using a scaling factor
step 2: compute the gradient with respect to the scaled loss
step 3: unscale the scaled gradient using the scaling factor
step 4: update model's parameters uisng the unscaled gradient

In [ ]:
step 1: partritioning
step 2: move to rank
step 3: move to device
step 4: ini local
step 5: step
step 6: broadcast
step 7: sync

In [ ]:
main worker > worker > task > cuda stream

In [ ]:
microbatch_idx, partrition_idx

kubernetes can't restore the last state of a failed node. it can only restart a pod with its initial configuration

In [ ]:
m+n-1

In [ ]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

In [2]:
n_cycles = 1_000_000

In [ ]:
start.record()
torch.cuda._sleep(n_cycles)
end.record()

In [ ]:
cycle_per_ms = n_cycles / end.elapsed_time(start)

In [ ]:
def cuda_sleep(seconds):
    ms = seconds * 1_000
    n_cycles = cycle_per_ms * ms
    torch.cuda._sleep(n_cycles)

In [7]:
class f(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        return input
    
    @staticmethod
    def backward(ctx, grad_input):
        torch.distributed.all_reduce(grad_input)
        return grad_input

In [6]:
class g(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        world_size = torch.distributed.get_world_size()
        inputs = [torch.empty(input) for _ in range(world_size)]
        torch.distributed.all_gather(inputs, input)
        inputs = torch.cat(inputs, dim=-1)
        return inputs
    
    @staticmethod
    def backward(ctx, grad_input):
        rank = torch.distributed.get_rank()
        world_size = torch.distributed.get_world_size()
        dim_size = grad_input.shape[-1]
        dim_size_per_partrition = dim_size // world_size
        grad_chunks = torch.split(grad_input, dim_size_per_partrition, dim=-1)
        return grad_chunks[rank]

In [3]:
class ColumnParallelLinear(nn.Module):
    def __init__(self, input_size, output_size, world_size):
        super().__init__()
        self.input_size = input_size
        self.output_size_per_partrition = output_size // world_size
        
        self.weight = nn.Parameter(torch.empty(
            self.output_size_per_partrition,
            self.input_size
        ))
        self.bias = nn.Parameter(torch.empty(
            self.output_size_per_partrition
        ))
    
    def forward(self, input):
        input_parallel = f.apply(input)
        output_paralell = F.linear(input_parallel, self.weight, self.bias)
        output = g.apply(output_paralell)
        return output

In [ ]:
step 1: fp32 and fp16 of the weight
step 2: forward and backward pass using fp16
step 3: cast the fp16 grad to fp32
step 4: 

In [ ]:
min(clock_idx+1, n_partritions)

In [ ]:
rank = torch.distributed.get_rank()

In [ ]:
if rank == 69:
    torch.distributed.isend(x, dest=42)
elif rank == 42:
    torch.distributed.irecv(
        tensor_will_be_received_data,
        src=69
    )

In [ ]:
clock_idx+1-n_microbatches

### MLE

In [ ]:
docker images

In [8]:
from typing import TypedDict

In [9]:
class NutritionInformation(TypedDict):
    value: int
    unit: str

In [10]:
class RecipeNutritionInformation(TypedDict):
    recipes_used: int
    calories: NutritionInformation
    carbs: NutritionInformation

In [13]:
from evidently.metric_preset import DataDriftPreset
from evidently.report import Report

In [14]:
report = Report(metrics=[
    DataDriftPreset()
])

In [ ]:
report.run(reference_data=reference, current_data=current)

In [15]:
from airflow.decorators import dag, task

In [ ]:
@dag(dag_id=dag_id, start_date=start_date)
def new_example():
    @task
    def get_name():
        pass
    
    @task
    def get_age():
        pass
    
    @task
    def greet(name, age):
        pass
    
    name = get_name()
    age = get_age()
    greet(name, age)

In [16]:
from sqlalchemy.orm import sessionmaker

In [ ]:
Session = sessionmaker(bind=bind)

In [ ]:
session = Session()

In [ ]:
session.add(user)

In [ ]:
session.commit()

In [ ]:
step 1: dvc add
step 2: dvc push
step 3: git add
s

### Engineering

In [17]:
import torch

### MechInterp

In [ ]:
tokens = model.to_tokens(text, prepend_bos=True)

In [ ]:
target_tokens = tokens[1:]

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
W_U_target_tokens = model.W_U[:, target_tokens]

In [18]:
from einops import einsum

In [ ]:
embed = cache["hook_embed"]

In [ ]:
embed_attributions = einsum(
    W_U_target_tokens,
    embed[:-1],
    "dim seq, seq dim -> sq"
)

In [19]:
def print_shape(module, input):
    print(input.shape)

In [ ]:
model.blocks[1].register_forward_pre_hook(print_shape)

In [ ]:
tokens = model.to_tokens(text)

In [ ]:
W_U = model.W_U

In [ ]:
target_tokens = tokens[1:]

In [ ]:
W_U_correct_tokens = W_U[:, target_tokens ]

In [ ]:
step 1: extract the correspond residual stream
step 2: unembedb
step 3: einsum

In [ ]:
unembed @ final_residual_stream
= unembed @ (embed + layer_1 + layer_2)
= unembed @ embed + unembed @ layer_1 + unembed @ layer_2

In [ ]:
step 1: the activation of embed
step 2: target tokens
step 3: 

In [20]:
import torch.distributed.rpc as rpc

In [21]:
futs = []

In [ ]:
for ob_rref in ob_rrefs:
    futs.append(rpc.rpc_async(
        to=ob_rref.owner(),
        func=ob_rref.rpc_sync().run
    ))

In [ ]:
for fut in futs:
    fut.append()

In [ ]:
def compute_cross_entropy(logits, targets):
    log_probs = F.log_softmax(logits, dim=-1)
    return -log_probs[torch.arange(target.shape[-1]), targets].mean()

In [ ]:
_, cache = model.run_with_cache(tokens)

In [ ]:
W_U = model.W_U

In [ ]:
embed = cache["hook_embed"]

In [ ]:
target_tokens = tokens[1:]

In [ ]:
W_U_target_tokens = W_U[:, target_tokens]

In [22]:
from einops import einsum

In [ ]:
embed_attributions = einsum(
    W_U_target_tokens,
    embed[:-1],
    "embed seq, seq embed -> seq"
)

In [ ]:
step 1: extract the correspond residual stream
step 2: 

In [ ]:
worker's id
worker's name
worker info

In [23]:
import pytest

In [ ]:
@pytest.fixture
def book():
    return Book("zoo")

In [ ]:
def test_book(book):
    assert book.name = "zoo"

In [ ]:
logits = unembed @ final_residual_stream
= unembed @ (embed + layer_1 + layer_2) 
= unembed @ embed + unembed @ layer_1 + unembed @ layer_2

In [ ]:
ob_rref.owner()

In [24]:
import torch.multiprocessing as mp

In [25]:
import os

In [27]:
def run_worker(rank, world_size):
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "29500"
    
    rpc.init_rpc(
        rank=rank,
        world_size=world_Size
    )

In [ ]:
for rank in range(world_size):
    process = mp.Process(
        target=run_worker,
        args=(rank, world_size)
    )
    process.start()

In [ ]:
x[:, [1, 3]]

In [ ]:
step 1: the output of the head
step 2: the unembed of the target tokens
step 3: @

In [ ]:
step 1: unembed[correct_tokens]
step 2: target residual stream
step 3: @

In [ ]:
def calculate_loss_of_soft_v(states, actions, action_probs):
    entropy = -action_probs.log()
    q_preds = q_function(states, actions)
    v_target = v_function(states)
    v_pred = (q_preds + entropy).mean()
    return (v_target - v_pred).pow(2).mean()